In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import tqdm
import pickle
from collections import Counter

%matplotlib notebook

In [2]:
df_train = pd.read_csv('data/train.csv', sep='\t')

In [7]:
df_train.columns

Index(['id', 'name', 'description', 'target'], dtype='object')

In [9]:
def get_symbols_dict(t):
    letters = [x for x in t]
    return Counter(letters)

all_names = df_train.name.tolist()
all_names_dicts = []

def calc_all_dicts():
    for a in tqdm.tqdm(all_names):
        all_names_dicts.append(get_symbols_dict(a))

In [10]:
calc_all_dicts()

100%|███████████████████████████████| 200000/200000 [00:03<00:00, 60346.58it/s]


In [19]:
all_names_dicts = pd.DataFrame(all_names_dicts)
all_names_dicts.fillna(0, inplace=True)
all_names_dicts.to_pickle('raws/all_names_dicts.pickle')

In [20]:
all_names_dicts = pd.concat([all_names_dicts, df_train.target], axis=1)

(200000, 195)

посмотрим что будет с обучением

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, log_loss, f1_score

X = all_names_dicts.drop(['target'], axis=1)
y = all_names_dicts.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size=.3)
model = RandomForestClassifier(random_state=1, n_estimators=50)
model.fit(X_train, y_train)

print('score:\t\t', model.score(X_test, y_test))
print('auc:\t\t', roc_auc_score(y_test, model.predict(X_test)) )
print('log_loss:\t', log_loss( y_test, model.predict(X_test) ))
print('f1:\t\t', f1_score( y_test, model.predict(X_test) ))

score:		 0.9447
auc:		 0.944936944794
log_loss:	 1.91001933538
f1:		 0.941059437951


In [38]:
from sklearn.model_selection import cross_val_score, KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=1)

cross_val_score(model, X, y, scoring='roc_auc', cv=kfold).mean()

0.98386591387809386

In [39]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

cross_val_score(model, X, y, scoring='f1', cv=kfold).mean()

0.94330297833323551